In [1]:
import os
import datetime

import IPython
import IPython.display
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler

d = pd.read_csv('./datapoints.csv')
d = d[d['name'].str.contains("BLOK61")]

# Converting the dt column to datetime object
d['timestamp'] = pd.to_datetime(d.pop('timestamp'), format='%Y-%m-%d %H:%M:%S')

d.sort_values('timestamp', inplace=True)


In [2]:
# Extracting the hour of day
d["hour"] = [x.hour for x in d["timestamp"]]
# Creating the cyclical daily feature
d["day_cos"] = [np.cos(x * (2 * np.pi / 24)) for x in d["hour"]]
d["day_sin"] = [np.sin(x * (2 * np.pi / 24)) for x in d["hour"]]

d.head()

,name,attribute_name,value,timestamp,hour,day_cos,day_sin
45023,Philitelaan BLOK61,occupied,3.0,2021-04-05 02:00:50.497,2,0.866025,0.5
45025,Philitelaan BLOK61,occupied,3.0,2021-04-05 02:01:57.041,2,0.866025,0.5
45027,Philitelaan BLOK61,occupied,3.0,2021-04-05 02:03:02.028,2,0.866025,0.5
45029,Philitelaan BLOK61,occupied,3.0,2021-04-05 02:04:07.265,2,0.866025,0.5
45031,Philitelaan BLOK61,occupied,3.0,2021-04-05 02:05:15.001,2,0.866025,0.5


In [3]:
# Extracting the timestamp from the datetime object
d["timestamp"] = [x.timestamp() for x in d["timestamp"]]
# Seconds in day
s = 24 * 60 * 60
# Seconds in year
year = (365.25) * s
d["month_cos"] = [np.cos((x) * (2 * np.pi / year)) for x in d["timestamp"]]
d["month_sin"] = [np.sin((x) * (2 * np.pi / year)) for x in d["timestamp"]]


In [4]:
def create_X_Y(ts: np.array, lag=1, n_ahead=1, target_index=0) -> tuple:
    """
    A method to create X and Y matrix from a time series array for the training of
    deep learning models
    """
    # Extracting the number of features that are passed from the array
    n_features = ts.shape[1]

    # Creating placeholder lists
    X, Y = [], []

    if len(ts) - lag <= 0:
        X.append(ts)
    else:
        for i in range(len(ts) - lag - n_ahead):
            Y.append(ts[(i + lag):(i + lag + n_ahead), target_index])
            X.append(ts[i:(i + lag)])

    X, Y = np.array(X), np.array(Y)

    # Reshaping the X array to an RNN input shape
    X = np.reshape(X, (X.shape[0], lag, n_features))

    return X, Y
